In [ ]:
! pip3 install deepeval python-dotenv boto3 aiobotocore botocore pandas

In [25]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
# import boto3
from dotenv import load_dotenv, find_dotenv
import os
from deepeval.models import GPTModel #AmazonBedrockModel
import pandas as pd

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRelevancyMetric

In [4]:
emb_model = OllamaEmbeddings(model="bge-m3:latest", num_thread=4)

In [ ]:
vectorstore = Chroma(
    embedding_function=emb_model,
    collection_name='guides',
    persist_directory="../data/chroma_knowledge_base"
)

In [6]:
# # === AWS Configuration === #
# COGNITO_REGION = os.getenv("COGNITO_REGION")
# BEDROCK_REGION = os.getenv("BEDROCK_REGION")
# MODEL_ID1 = os.getenv("MODEL_ID1")
# MODEL_ID2 = os.getenv("MODEL_ID2")
# IDENTITY_POOL_ID = os.getenv("IDENTITY_POOL_ID")
# USER_POOL_ID = os.getenv("USER_POOL_ID")
# APP_CLIENT_ID = os.getenv("APP_CLIENT_ID")
# USERNAME = os.getenv("USERNAME")
# PASSWORD = os.getenv("PASSWORD")

In [7]:
# # === Helper: Get AWS Credentials === #
# def get_credentials(username, password):
#     idp_client = boto3.client("cognito-idp", region_name=COGNITO_REGION)
#     response = idp_client.initiate_auth(
#         AuthFlow="USER_PASSWORD_AUTH",
#         AuthParameters={"USERNAME": username, "PASSWORD": password},
#         ClientId=APP_CLIENT_ID,
#     )
#     id_token = response["AuthenticationResult"]["IdToken"]

#     identity_client = boto3.client("cognito-identity", region_name=COGNITO_REGION)
#     identity_response = identity_client.get_id(
#         IdentityPoolId=IDENTITY_POOL_ID,
#         Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
#     )

#     creds_response = identity_client.get_credentials_for_identity(
#         IdentityId=identity_response["IdentityId"],
#         Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
#     )

#     return creds_response["Credentials"]

In [8]:
# credentials = get_credentials(USERNAME, PASSWORD)

In [9]:
# os.environ["AWS_ACCESS_KEY_ID"]=credentials.get('AccessKeyId')
# os.environ["AWS_SECRET_ACCESS_KEY"]=credentials.get('SecretKey')
# os.environ["AWS_SESSION_TOKEN"]=credentials.get('SessionToken')

In [ ]:
eval_dataset = pd.read_csv("../data/datasets/canvas_community_forum.csv")

In [11]:
eval_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   input            19 non-null     object
 1   expected_output  19 non-null     object
dtypes: object(2)
memory usage: 436.0+ bytes


In [12]:
eval_dataset.head()

,input,expected_output
0,"Hey, I'm a bit unsure about a message I sent t...",You sent it to yourself only. It will only sen...
1,I am a new student and my course starts tomorr...,"Unfortunately, we won't be able to directly he..."
2,コース登録した授業のダッシュボードを消すにはどうすればいいですか？,I believe this is your concern? How to delete ...
3,Today I was trying to keep up with my work in ...,It can depend on the teacher and how they orga...
4,How do I find my unoffical transcripts in canv...,Since this question appears to be school-speci...


In [13]:
# model = AmazonBedrockModel(
#     model_id=MODEL_ID1,
#     region_name=BEDROCK_REGION,
#     generation_kwargs={
#         "temperature": 0,
#         "maxTokens":500
#     }
# )

model = GPTModel(
    model="o4-mini",
    temperature=1
)

In [ ]:
os.environ["DEEPEVAL_RESULTS_FOLDER"]="../data/eval_results"

In [15]:
metric = ContextualRelevancyMetric(
    threshold=0.7,
    model=model,
    include_reason=True
)

In [16]:
def retrieve(query):
    retrieved_docs = vectorstore.as_retriever(search_kwargs={"k":3}).invoke(query)

    return list(map(lambda doc: doc.page_content, retrieved_docs))

In [ ]:
test_cases = []

for input, expected_output in eval_dataset.values.tolist(): 

    actual_output = ""
    retrieval_context = retrieve(input)


    test_case = LLMTestCase(
        input=input,
        actual_output=actual_output,
        retrieval_context=retrieval_context,
        expected_output=expected_output
    )

    test_cases.append(test_case)

In [18]:
total_test_cases = len(test_cases)

In [ ]:
! pip3 install ipywidgets

In [ ]:
# import textwrap
# pp = textwrap.TextWrapper(width=100)
# test_cases_successful = 0

# for case, test_case in enumerate(test_cases, start=1):
#     metric.measure(test_case)
#     test_cases_successful += int(metric.is_successful())
#     print(f"Case {case} | Score: {metric.score}")
#     print(pp.fill(f"Reason: {metric.reason}"))
#     print("-"*50)

In [27]:
evaluate(test_cases=test_cases, metrics=[metric])

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using o4-mini, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ❌ Contextual Relevancy (score: 0.6666666666666666, threshold: 0.7, strict: False, evaluation model: o4-mini, reason: The score is 0.67 because while relevant statements like "Some instructors will allow you to retake a quiz" and "If you can retake the quiz, you will see Take the Quiz Again button" address resetting, most context only covers resuming (e.g., "You can resume a quiz you previously started") and navigation instructions, not quiz resets., error: None)

For test case:

  - input: Hello, so I was checking out a course that I am taking and I clicked on a quiz and it started. Is there anyway that someone could reset it? Please help! The quiz is due Sunday!
  - actual output: 
  - expected output: Students are not able to reset their own quiz progress. Please reach out to your instructor for assistance.
  - context: None
  - retrieval context: ['Document Title: How do I resume a quiz that I already started taking\n\nHow do I resume a quiz that I already sta

⚠ WARNING: No hyperparameters logged.
» ]8;id=140205;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

Results saved in ./data as 20251029_232511

✓ Evaluation completed 🎉! (time taken: 43.5s | token cost: 0.5490419 USD)
» Test Results (19 total tests):
   » Pass Rate: 5.26% | Passed: 1 | Failed: 18

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_14', success=False, metrics_data=[MetricData(name='Contextual Relevancy', threshold=0.7, success=False, score=0.6666666666666666, reason='The score is 0.67 because while relevant statements like "Some instructors will allow you to retake a quiz" and "If you can retake the quiz, you will see Take the Quiz Again button" address resetting, most context only covers resuming (e.g., "You can resume a quiz you previously started") and navigation instructions, not quiz resets.', strict_mode=False, evaluation_model='o4-mini', error=None, evaluation_cost=0.021145300000000002, verbose_logs='Verdicts:\n[\n    {\n        "verdicts": [\n            {\n                "statement": "You can resume a quiz you previously started.",\n                "verdict": "no",\n                "reason": "The statement \'You can resume a quiz you previously started\' does not address resetting a quiz; it only explains resuming."\n            },\n            {

Possible reasons as to why we are scoring low on Context Relevancy?

- the user input may contain noise (painc, urgency + unnessary details) which is being captured by the embedding model when retrieving relevant document and maybe leading to retrieving the incorrect docs
    - possible remedy: **query-rewriting**; use llm to pin-point the root problem

- Each retrieved Document consists <= 4200 characters approx 1024 tokens, choosing a large chunk size was an intentional decision to preserve the context. This may be negatively impacting the Context Relevancy metric; huge doc ---> lots of statements generated (most of which may not be relevant to the user query)
    - possible remedy: **contextual compression**; this technique aims to improve the relevance and conciseness of retrieved information by compressing and extracting the most pertinent parts of documents in the context of a given query.